<a href="https://colab.research.google.com/github/kfir1g/AI-Project/blob/master/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


#Initialize tensorflow-GPU

In [2]:
!pip install tensorflow-gpu
import tensorflow as tf 
#test111

     |████████████████████████████████| 377.0MB 51kB/s 


##This part makes sure that we are working on GPU in runtime

In [0]:

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at : {}'.format(device_name))  

Found GPU at : /device:GPU:0


#Data Preprocessing

##importing Libraries and Data

##Import the data
data is the results of each tree on each line of the testing data + the actual value

In [0]:
#from google.colab import files
#file = files.upload()
import numpy as np
import pandas as pd
filename = #path of the file to be trained
dataset = pd.read_csv("/content/gdrive/" + filename)

##Getting the dependent and independent variables

splits the information to results from tree and actual result

In [0]:
X = dataset.iloc[:,1:-1].values 
y = dataset.iloc[:,-1].values

In [0]:
MAX_PICKUP_BIKES = y.max() +1 #Todo: check what is the max number
NUM_OF_INPUTS = X.shape[1]

##Splitting the dataset into training set and Test set 

Splits the information to training and test for the NN, test_size can be changed

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)


##Feature scaling - not needed for trees

In [0]:
#Not needed for sure
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_Y = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)



#Builed the NN

##Parameters for NN
softmax - output neuron that got the highest value
Epoch - how many time will go over all the data
Batch_size - update weights every batch inputs
Can add more layers or change the numbers

In [0]:
FIRST_LAYAR = 200
SECONED_LAYAR = 200
OUTPUT_ACTIVATION_FUNC = 'softmax'
OPTIMIZER = 'adam' #'rmsprop' #stochastich gradient descent
NUM_EPOCH = 100
BATCH_SIZE = 75

First layer has to have num_of_inputs
Last layer has to have that activation
We can add more layers, by copying the add option and changing options

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
classifier = Sequential()

classifier.add(Dense(output_dim = FIRST_LAYAR  , init = 'uniform', activation = 'relu', input_dim = NUM_OF_INPUTS))
classifier.add(Dense(output_dim = SECONED_LAYAR, init = 'uniform', activation = 'relu' ))
classifier.add(Dense(output_dim =MAX_PICKUP_BIKES, init = 'uniform', activation = OUTPUT_ACTIVATION_FUNC ))
 

#Define Optimizer and Loss function 

Change the values to change the NN

In [0]:
classifier.compile(optimizer = OPTIMIZER , loss = 'sparse_categorical_crossentropy',metrics = ['accuracy']) 

#Train the data 

fit is the training, after that saves the weights

In [0]:
classifier.fit(X_train, Y_train, batch_size = BATCH_SIZE , nb_epoch = NUM_EPOCH)
from keras.models import load_model
classifier.save_weights('/content/gdrive/My Drive/AIProj/my_model_weights.h5')

#Extract the predctions 


1.   getting the predection from the classifier
2.   extracting the argmax which is the number of picups fo reach input
3.  checking while using the test vector our results




In [0]:
Y_pred = classifier.predict(X_test)
Y_get = []
for i in range(len(Y_pred)):
  Y_get.append(Y_pred[i].argmax())
  

In [0]:
total = 0 
correct = 0 
wrong = 0 
for i in range(len(Y_pred)):
  total = total +1
  if(Y_get[i] == Y_test[i]):
     correct = correct + 1
  else:
    wrong = wrong + 1
    
print("Total "+ str(total) )
print("Correct " + str(correct))
print("Wrong " + str(wrong))
print("acc:" + str(float(correct)/total) )
    

Total 30138
Correct 9819
Wrong 20319
acc:0.3258013139558033


#Load Wights 
To Load wights you need to create the model again and use the classifier to predict  - need to be tested

In [0]:

MAX_PICKUP_BIKES = y.max() +1 #Todo: check what is the max number
NUM_OF_INPUTS = X.shape[1]
FIRST_LAYAR = 200
SECONED_LAYAR = 200
OUTPUT_ACTIVATION_FUNC = 'softmax'
OPTIMIZER = 'rmsprop' #'adam' #stochastich gradient descent
NUM_EPOCH = 100
BATCH_SIZE = 75

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
classifier = Sequential()

classifier.add(Dense(output_dim = FIRST_LAYAR  , init = 'uniform',activation ='relu',input_dim = NUM_OF_INPUTS ))
classifier.add(Dense(output_dim = SECONED_LAYAR, init = 'uniform', activation = 'relu' ))
classifier.add(Dense(output_dim =355, init = 'uniform', activation = OUTPUT_ACTIVATION_FUNC ))
classifier.load_weights('my_model_weights.h5')

#How to predict one input: 

input_vector = #put input vector you want to test
Y_pred = classifier.predict(input_vector)
pickup_predection = Y_pred[0].argmax()


#Evaluating the ANN 

##Using K-Fold Cross Validation 
The main idea is dividing to 10 folds and taking the mean and variance, the mean is the actual result of our ANN and the variance will indicate if we are overfitting
NN needs to be the same as before

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
def build_classifier():
  classifier = Sequential()

  classifier.add(Dense(output_dim = FIRST_LAYAR  , init = 'uniform',activation ='relu',input_dim = NUM_OF_INPUTS ))
  classifier.add(Dense(output_dim = SECONED_LAYAR, init = 'uniform', activation = 'relu' ))
  classifier.add(Dense(output_dim = 355, init = 'uniform', activation = OUTPUT_ACTIVATION_FUNC ))
  classifier.compile(optimizer = 'adam'  , loss = 'sparse_categorical_crossentropy',metrics = ['accuracy'])
  return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 75, nb_epoch = 250 )
accuracies = cross_val_score(estimator = classifier, X = X_train, y = Y_train, cv =10, n_jobs = -1)# cv num of folds , n_jobs =-1 usingall the GPU
#Accuracies is a vector with all the #folds results(in our case 10) 
#The real Result is the mean of this vector, also we need to check for hight or low variance to know if our results are good
mean = accuracies.mean()
variance = accuracies.std()
variance
  

0.005668231143304357

In [0]:
mean

0.32813215512885086

#Improving the ANN

##Droput Regularization to reduce overfitting if needed 
While Getting big variance, meaning the model was doing well on some and than really bad on others, so he is overfitting on the training.
This startagy will help fixing it. 

##Tuning The ANN - choose the best param from all the options


In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
def build_classifier(optimizer,first, seconed ):
  classifier = Sequential()

  classifier.add(Dense(output_dim = first  , init = 'uniform',activation ='relu',input_dim = NUM_OF_INPUTS ))
  classifier.add(Dense(output_dim = seconed, init = 'uniform', activation = 'relu' ))
  classifier.add(Dense(output_dim = 355, init = 'uniform', activation = OUTPUT_ACTIVATION_FUNC ))
  classifier.compile(optimizer = optimizer  , loss = 'sparse_categorical_crossentropy',metrics = ['accuracy'])
  return classifier

classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [25, 32 ],'nb_epochs': [250, 500], 'optimizer' : ['adam', 'rmsprop'], 'first': [100,200], 'seconed':[100,200]}
grid_search = GridSearchCV(estimator = classifier, param_grid = parameters, scoring = 'accuracy', cv = 10)  
grid_serach = grid_search.fit(X_train, Y_train)                     
best_parameters = grid_serach.best_params_
best_accuracy = grid_search.best_score_